### Introduction

We are going to present the `anytree` package that we use for the `tree_maker`.
Please find in the 001_example a proper `tree_maker` example.

In [1]:
import anytree

In [2]:
from anytree import AnyNode, RenderTree
root = AnyNode(name="root")
#first generation
[AnyNode(name=f"{ii}", parent=root) for ii in range(2)]
for aa, bb in enumerate(root.children):
    #second generation
    [AnyNode(name=f"{bb.name}{ii}", parent=bb) for ii in range(3)]
    for aa, bb in enumerate(bb.children):
        #third generation
        [AnyNode(name=f"{bb.name}{ii}", parent=bb) for ii in range(2)]
        #...

#print(RenderTree(root))
for pre, _, node in RenderTree(root, style=anytree.render.ContRoundStyle()):
    print(f"{pre}{node.name}")

root
├── 0
│   ├── 00
│   │   ├── 000
│   │   ╰── 001
│   ├── 01
│   │   ├── 010
│   │   ╰── 011
│   ╰── 02
│       ├── 020
│       ╰── 021
╰── 1
    ├── 10
    │   ├── 100
    │   ╰── 101
    ├── 11
    │   ├── 110
    │   ╰── 111
    ╰── 12
        ├── 120
        ╰── 121


In [3]:
from anytree.exporter import DotExporter
DotExporter(root).to_picture('tree.png')

In [4]:
# Concept of children of a node
[ii.name for ii in  root.children]

['0', '1']

In [5]:
# Concept of parent of a node
# There is a single parent. Is it a limitation? 
# Tree is a "limited" Directed Acycling Graph Directed acyclic graph 
(https://en.wikipedia.org/wiki/Directed_acyclic_graph)
my_node=root.children[1].children[0]
my_node.name

'10'

In [8]:
my_node.parent.name

'1'

In [9]:
# Concept of ancestors
[ii.name for ii in  my_node.ancestors]

['root', '1']

In [10]:
# Concept of descendants
[ii.name for ii in  my_node.descendants]

['100', '101']

In [11]:
# Concept of sibling
[ii.name for ii in  my_node.siblings]

['11', '12']

In [12]:
# Concept of path
[ii.name for ii in  my_node.path]

['root', '1', '10']

In [13]:
# We can link attribute to a node
my_node.root.name

'root'

In [14]:
# Concept of height of a node
root.height

3

In [15]:
# Concept of depth of a node
# This is important to define a "generation"
my_node.depth

2

In [16]:
# Concept of leaves of a node
[ii.name for ii in  root.leaves]

['000',
 '001',
 '010',
 '011',
 '020',
 '021',
 '100',
 '101',
 '110',
 '111',
 '120',
 '121']

In [17]:
# Search in node and
# select all nodes of a given node depth
# VERY IMPORTANT
[ii.name for ii in anytree.search.findall(root, filter_=lambda node: node.depth==2)]

['00', '01', '02', '10', '11', '12']

In [18]:
# Walk in a tree
w = anytree.walker.Walker()
[ii.name for ii in w.walk(root, root.leaves[0])[-1]]

['0', '00', '000']

In [19]:
# Save the tree in a yaml
import yaml
from anytree import AnyNode
from anytree.exporter import DictExporter
from anytree.importer import DictImporter

dct = DictExporter().export(root)

with open("tree.yaml", "w") as file:  
    yaml.dump(dct, file)

In [20]:
# Load the tree from a  yaml
with open("tree.yaml", "r") as file:
    root = DictImporter().import_(yaml.load(file, Loader=yaml.FullLoader))
root = DictImporter().import_(dct)
for pre, _, node in RenderTree(root, style=anytree.render.ContRoundStyle()):
    print(f"{pre}{node.name}")

root
├── 0
│   ├── 00
│   │   ├── 000
│   │   ╰── 001
│   ├── 01
│   │   ├── 010
│   │   ╰── 011
│   ╰── 02
│       ├── 020
│       ╰── 021
╰── 1
    ├── 10
    │   ├── 100
    │   ╰── 101
    ├── 11
    │   ├── 110
    │   ╰── 111
    ╰── 12
        ├── 120
        ╰── 121


In [21]:
%%timeit
# performance 400x30 jobs
from anytree import AnyNode, RenderTree
root = AnyNode(name="root")
#first generation
[AnyNode(name=f"_{ii:03d}", parent=root) for ii in range(400)]
for aa, bb in enumerate(root.children):
    #second generation
    [AnyNode(name=f"__{ii:03d}", parent=bb) for ii in range(30)]

122 ms ± 34.2 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [22]:
%%timeit
[ii.name for ii in anytree.search.findall(root, filter_=lambda node: node.depth==2)]

88.5 µs ± 4.5 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


We can define a generic class (`JobNode`) that inherits from `anytree` object.

In [23]:
from anytree import NodeMixin, RenderTree
class JobNodeBase(object):  # Just an example of a base class
    name = 'my_name'
    path = 'my_path'
    #def __repr__(self):
    #        return self.name    
    def __str__(self):
            return self.name
    def run(self):
        pass
    def clone(self):
        pass
    def mutate(self):
        pass

class JobNode(JobNodeBase, NodeMixin):  # Add Node feature
    def __init__(self, name, length, width, parent=None, children=None):
        super(JobNodeBase, self).__init__()
        self.name = name
        self.length = length
        self.width = width
        self.parent = parent
        if children:  # set children only if given
            self.children = children


In [24]:
root = JobNode('root', length=1, width=2, parent=None)
#first generation
[JobNode(name=f"{ii}", parent=root, length=1, width=2) for ii in range(2)]
for aa, bb in enumerate(root.children):
    #second generation
    [JobNode(name=f"{bb.name}{ii}", parent=bb, length=1, width=2) for ii in range(3)]

for pre, _, node in RenderTree(root, style=anytree.render.ContRoundStyle()):
    print(f"{pre}{node.name}")


root
├── 0
│   ├── 00
│   ├── 01
│   ╰── 02
╰── 1
    ├── 10
    ├── 11
    ╰── 12
